In [ ]:
import pandas as pd
import numpy as np
import sklearn
from numpy.linalg import eig
from sklearn.model_selection import train_test_split

In [1]:
def pca(train_df):
    """
    Gives the Principle Components.

    :train_df: numpy 2D array, data to reduce the dimensionality.

    :return: numpy 2D array, Soreted eigen vectors.
    :return: numpy 2D array, varience captured by components.
    """

    #computing the covarience matrix
    covarience_mat = np.cov(train_df, rowvar=False)
    print(covarience_mat.shape)
    #computing eigen values and eigen vectors
    eigen_val, eigen_vec = eig(covarience_mat)
    index = np.argsort(eigen_val)[::-1]
    #sorting the eigen values
    sorted_eigen_val = eigen_val[index]
    varience_captured = sorted_eigen_val/np.sum(sorted_eigen_val)
    #sort the eigenvectors according to index
    sorted_eigen_vec = eigen_vec[:,index]
    #selecting the 70 components
    
    # principle_comp = sorted_eigen_vec[:,0:principle_components]
    # principle_comp.shape 

    return  sorted_eigen_vec, varience_captured


In [ ]:
def mda(X_train, comp):
    # compute the class means
    means = []
    data = []
    K = X_train['labels'].nunique()
    for k in range(K):
        train = (X_train[X_train['labels']==k].values)
        train_data = train[:,0:-1]
        data.append(train_data)

        train_mean = np.mean(train_data,axis=0)
        means.append(train_mean)
    M=means

    # compute the scatter matrices
    Sw = np.zeros((X_train.shape[1]-1, X_train.shape[1]-1))
    for k in range(K):
        Sw += np.dot((data[k]-M[k]).T, (data[k]-M[k]))

    # Calculate between-class scatter matrix
    X = X_train.values
    overall_mean = np.mean(X[:,0:-1], axis=0)
    S_b = np.zeros((X_train.shape[1]-1, X_train.shape[1]-1))
    for c in range(K):
        n = data[c].shape[0]
        diff = (M[c] - overall_mean).reshape(-1, 1)
        S_b += n * np.dot(diff, diff.T)

    # Calculate eigenvectors and eigenvalues of S_w^-1 S_b
    S_w_inv = np.linalg.inv(Sw)
    eigvals, eigvecs = np.linalg.eig(np.dot(S_w_inv, S_b))

    index = np.argsort(eigvals)[::-1]
    #sorting the eigen values
    sorted_eigen_val = eigvals[index]
    varience_captured = sorted_eigen_val/np.sum(sorted_eigen_val)
    #sort the eigenvectors according to index
    sorted_eigen_vec = eigvecs[:,index]

    components = sorted_eigen_vec[:,:comp]
    
    return components